<a href="https://colab.research.google.com/github/flahbocchino/farmtech-fase7-grupoFlavia-/blob/main/fase7_notebook/FarmTech_Fase7_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/flahbocchino/farmtech-fase7-grupoFlavia-.git
%cd farmtech-fase7-grupoFlavia-


Cloning into 'farmtech-fase7-grupoFlavia-'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 76 (delta 21), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (76/76), 627.66 KiB | 7.56 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/farmtech-fase7-grupoFlavia-


In [2]:
!pip install streamlit pyngrok pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 118.8 MB/s eta 0:00:00


In [4]:
!git clone https://github.com/flahbocchino/farmtech-fase7-grupoFlavia-.git
%cd farmtech-fase7-grupoFlavia-


Cloning into 'farmtech-fase7-grupoFlavia-'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 76 (delta 21), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (76/76), 627.66 KiB | 6.47 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/farmtech-fase7-grupoFlavia-/farmtech-fase7-grupoFlavia-


In [5]:
!pip install streamlit pandas
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 22 packages in 3s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

In [6]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &>/dev/null &


In [14]:
!lt --port 8501


your url is: https://little-mice-unite.loca.lt
/tools/node/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:12113 (check your firewall settings)
    at Socket.<anonymous> (/tools/node/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0


In [13]:
!curl https://ipv4.icanhazip.com


34.41.0.72


In [15]:
%%writefile app.py
import streamlit as st
import pandas as pd
import sqlite3
from pathlib import Path

# Caminho base do projeto (pasta onde o app.py fica)
BASE_DIR = Path(__file__).parent

# =========================
# Funções auxiliares – Fase 1
# =========================

def carregar_bases_fase1():
    """Carrega as bases da Fase 1 (talhões/insumos e meteorologia)."""
    df_talhoes = pd.read_csv(
        BASE_DIR / "fase1_base_dados" / "base_fase1_talhoes_insumos.csv"
    )
    df_meteo = pd.read_csv(
        BASE_DIR / "fase1_base_dados" / "base_fase1_com_meteo.csv"
    )
    return df_talhoes, df_meteo

def pagina_fase1():
    st.header("Fase 1 – Base de dados da fazenda")

    df_talhoes, df_meteo = carregar_bases_fase1()

    st.subheader("Talhões e insumos")
    st.dataframe(df_talhoes)

    st.subheader("Dados meteorológicos combinados")
    st.dataframe(df_meteo)

    st.info(
        "Esses dados servem de base para as demais fases (IoT, predição e visão computacional)."
    )

# =========================
# Funções auxiliares – Fase 2 (banco relacional)
# =========================

def pagina_fase2():
    st.header("Fase 2 – Banco de dados relacional (SQLite)")

    db_path = BASE_DIR / "fase2_banco_relacional" / "farmtech.db"
    if not db_path.exists():
        st.error(f"Arquivo {db_path.name} não encontrado na pasta fase2_banco_relacional.")
        return

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Lista de tabelas
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tabelas = [row[0] for row in cursor.fetchall()]

    st.subheader("Tabelas disponíveis")
    st.write(tabelas)

    # Permite o usuário escolher uma tabela para visualizar
    if tabelas:
        tabela_escolhida = st.selectbox("Escolha uma tabela para visualizar:", tabelas)
        if tabela_escolhida:
            df = pd.read_sql_query(f"SELECT * FROM {tabela_escolhida} LIMIT 100;", conn)
            st.dataframe(df)

    conn.close()

# =========================
# Funções auxiliares – Fase 3 (IoT / leituras de sensor)
# =========================

def carregar_ultimas_leituras_fase3():
    """Lê as últimas leituras de sensores gravadas no banco da Fase 3."""
    # o arquivo pode ter sido salvo como farmtech.db ou farmtech (1).db
    db_path = BASE_DIR / "fase3_iot_esp32" / "farmtech.db"
    if not db_path.exists():
        db_path = BASE_DIR / "fase3_iot_esp32" / "farmtech (1).db"

    if not db_path.exists():
        return None

    conn = sqlite3.connect(db_path)
    try:
        # Ajuste o nome da tabela abaixo para o nome real que você usou na Fase 3
        df = pd.read_sql_query(
            "SELECT * FROM leituras_sensores ORDER BY timestamp DESC LIMIT 50;",
            conn,
        )
    except Exception:
        df = None
    finally:
        conn.close()
    return df

def pagina_fase3():
    st.header("Fase 3 – Leituras de sensores (IoT / ESP32)")

    df = carregar_ultimas_leituras_fase3()
    if df is None:
        st.warning(
            "Não foi possível carregar o banco da Fase 3 ou a tabela 'leituras_sensores'. "
            "Verifique o nome do banco/tabela."
        )
        return

    st.subheader("Últimas leituras de sensores")
    st.dataframe(df)

    st.info(
        "Essas leituras poderiam ser usadas para acionar irrigação automática "
        "e também para disparar alertas na AWS (Fase 5)."
    )

# =========================
# Funções auxiliares – Fase 6 (visão computacional)
# =========================

def carregar_resultados_visao():
    """Carrega o CSV gerado na Fase 6 com o resumo das detecções por imagem."""
    csv_path = BASE_DIR / "fase6_visao_computacional" / "fase6_resultados_visao.csv"
    if not csv_path.exists():
        return None
    df = pd.read_csv(csv_path)
    return df

def enviar_alerta_aws(mensagem: str):
    """
    Envia alerta via AWS SNS.

    Espera que as credenciais AWS e o ARN do tópico estejam configurados em variáveis
    de ambiente: AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_DEFAULT_REGION,
    AWS_SNS_TOPIC_ARN.
    """
    try:
        import boto3
        import os

        topic_arn = os.environ.get("AWS_SNS_TOPIC_ARN")
        if not topic_arn:
            st.error("Variável de ambiente AWS_SNS_TOPIC_ARN não configurada.")
            return

        sns = boto3.client("sns", region_name=os.environ.get("AWS_DEFAULT_REGION", "us-east-1"))
        sns.publish(TopicArn=topic_arn, Message=mensagem, Subject="Alerta FarmTech – Visão Computacional")
        st.success("Alerta enviado para o tópico SNS configurado.")
    except Exception as e:
        st.error(f"Falha ao enviar alerta AWS SNS: {e}")

def pagina_fase6():
    st.header("Fase 6 – Visão computacional da lavoura")

    df = carregar_resultados_visao()
    if df is None:
        st.error("Arquivo fase6_resultados_visao.csv não encontrado na pasta fase6_visao_computacional.")
        return

    st.subheader("Resumo das detecções por imagem")
    st.dataframe(df)

    st.subheader("Análise de risco")
    # Exemplo simples: considera "risco" se houver qualquer objeto detectado
    if "objetos_detectados" in df.columns:
        total_com_objetos = (df["objetos_detectados"] > 0).sum()
        st.write(f"Imagens com algum objeto detectado: {total_com_objetos} de {len(df)}.")
    else:
        st.write("Coluna 'objetos_detectados' não encontrada no CSV, usando apenas visualização.")

    if st.button("Analisar e enviar alerta via AWS (Fase 5)"):
        if "objetos_detectados" in df.columns and (df["objetos_detectados"] > 0).any():
            mensagem = (
                f"Foram encontradas imagens com possíveis problemas na lavoura. "
                f"Total com objetos detectados: {(df['objetos_detectados'] > 0).sum()}."
            )
            enviar_alerta_aws(mensagem)
        else:
            st.info("Nenhum problema crítico detectado, nenhum alerta enviado.")

# =========================
# Função principal (menu da dashboard)
# =========================

def main():
    st.set_page_config(page_title="FarmTech – Fase 7", layout="wide")
    st.title("FarmTech – Fase 7: Consolidação do Sistema")

    menu = st.sidebar.radio(
        "Selecione a Fase:",
        (
            "Visão geral",
            "Fase 1 – Base de dados",
            "Fase 2 – Banco relacional",
            "Fase 3 – Leituras IoT",
            "Fase 6 – Visão computacional + AWS",
        ),
    )

    if menu == "Visão geral":
        st.subheader("Visão geral do projeto")
        st.markdown(
            """
            - **Fase 1**: organização dos dados de talhões, insumos e meteorologia.
            - **Fase 2**: modelagem e criação do banco relacional em SQLite.
            - **Fase 3**: leituras de sensores via ESP32 e gravação no banco.
            - **Fase 4**: (no projeto original) criação de modelos preditivos e dashboard de data science.
            - **Fase 5**: infraestrutura na AWS e serviço de alertas via SNS.
            - **Fase 6**: visão computacional para monitorar a lavoura.
            - **Fase 7**: esta dashboard integrando tudo em um único sistema.
            """
        )
    elif menu == "Fase 1 – Base de dados":
        pagina_fase1()
    elif menu == "Fase 2 – Banco relacional":
        pagina_fase2()
    elif menu == "Fase 3 – Leituras IoT":
        pagina_fase3()
    elif menu == "Fase 6 – Visão computacional + AWS":
        pagina_fase6()

if __name__ == "__main__":
    main()


Overwriting app.py
